In [9]:
from tensorflow import keras
from keras import layers
import os
import numpy as np
import matplotlib

In [13]:
original_dim = (256,256)
intermediate_dim = 64
latent_dim = 2

In [3]:
inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

from keras import backend as K

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')


In [4]:
# Create the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

In [18]:
from PIL import Image
subreddits = os.listdir("Data")
for subreddit in subreddits:
    subreddit_folder = os.path.join("Data", subreddit)
    image_paths = [os.path.join(subreddit_folder, filename) for filename in os.listdir(subreddit_folder) if ".csv" not in filename]
    for image_path in image_paths:
        image = Image.open(image_path)
        image.thumbnail(original_dim)
        # image.show()
        image_array = np.asarray(image).T
        print(image_array.shape)
        

(3, 166, 256)
(3, 174, 256)
(3, 203, 256)
(3, 140, 256)
(3, 256, 256)
(3, 163, 256)
(4, 256, 170)
(3, 256, 256)
(3, 256, 192)
(3, 256, 168)
(3, 192, 256)
(3, 256, 171)
(3, 256, 181)
(3, 256, 256)
(3, 256, 256)
(4, 256, 153)
(3, 181, 256)
(3, 192, 256)
(3, 185, 256)
(3, 192, 256)
(3, 211, 256)
(3, 192, 256)
(3, 150, 256)
(4, 256, 256)
(3, 256, 183)
(3, 256, 204)
(3, 192, 256)
(3, 194, 256)
(4, 250, 256)
(3, 192, 256)
(3, 256, 205)
(3, 192, 256)
(3, 256, 181)
(180, 256)
(3, 256, 256)
(3, 192, 256)
(3, 256, 222)
(4, 256, 192)
(3, 256, 256)
(3, 192, 256)
(3, 192, 256)
(3, 256, 256)
(3, 256, 256)
(3, 192, 256)
(3, 256, 184)
(4, 256, 256)
(3, 171, 256)
(3, 254, 256)
(3, 212, 256)
(4, 229, 256)
(4, 256, 249)
(3, 188, 256)
(3, 249, 256)
(3, 256, 205)
(3, 237, 256)
(4, 256, 177)
(3, 256, 256)
(3, 256, 256)
(3, 256, 146)
(3, 256, 256)
(3, 144, 256)
(3, 192, 256)
(3, 256, 192)
(3, 256, 161)
(3, 205, 256)
(3, 223, 256)
(3, 229, 256)
(3, 256, 171)
(3, 256, 225)
(3, 192, 256)
(3, 236, 256)
(4, 256, 